In [ ]:
import os
import numpy as np
import tensorflow as tf
from collections import defaultdict
from music21 import converter, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import classification_report
import keras_tuner as kt

import warnings
warnings.filterwarnings('ignore')

# random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


## Data Pre-processing

In [4]:
dataset_root = '../Composer_Dataset/NN_midi_files_extended'
splits = ['train', 'dev', 'test']

# Inspect dataset
composer_counts = defaultdict(int)
total_files = 0

for split in splits:
    split_path = os.path.join(dataset_root, split)
    for composer in os.listdir(split_path):
        composer_dir = os.path.join(split_path, composer)
        if os.path.isdir(composer_dir):
            count = len([f for f in os.listdir(composer_dir) if f.endswith('.mid')])
            composer_counts[composer] += count
            total_files += count

print(f"Total MIDI files: {total_files}")
print(f"Composers: {list(composer_counts.keys())}")
print("Files per composer:")
for composer, count in composer_counts.items():
    print(f"   - {composer}: {count}")

Total MIDI files: 439
Composers: ['mozart', 'chopin', 'handel', 'byrd', 'schumann', 'mendelssohn', 'hummel', 'bach', 'bartok']
Files per composer:
   - mozart: 49
   - chopin: 49
   - handel: 49
   - byrd: 50
   - schumann: 44
   - mendelssohn: 49
   - hummel: 50
   - bach: 50
   - bartok: 49


## Feature Extraction

In [5]:
def extract_sequence(midi_path):
    midi = converter.parse(midi_path)
    sequence = []
    for el in midi.flat.notes:
        if isinstance(el, note.Note):
            sequence.append(str(el.pitch))
        elif isinstance(el, chord.Chord):
            sequence.append('.'.join(str(n) for n in el.normalOrder))
    return sequence

X = []
y = []

for split in splits:
    split_path = os.path.join(dataset_root, split)
    for composer in os.listdir(split_path):
        composer_dir = os.path.join(split_path, composer)
        if os.path.isdir(composer_dir):
            for fname in os.listdir(composer_dir):
                if fname.endswith('.mid'):
                    path = os.path.join(composer_dir, fname)
                    seq = extract_sequence(path)
                    X.append(seq)
                    y.append(composer)

print(f"Extracted {len(X)} sequences with labels.")

Extracted 439 sequences with labels.


## Model Building

In [ ]:

input_dim = 200 
output_dim = len(set(y))  # number of composers
seq_length = 60  # 

def build_lstm():
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128, input_length=seq_length),
        LSTM(128, return_sequences=True),
        LSTM(64),
        Dense(64, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn():
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128, input_length=seq_length),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Conv1D(128, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Model Training

In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

## Model Evaluation

In [ ]:

y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=list(set(y))))

## Model Optimization

In [ ]:


def build_tuned_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=128, input_length=seq_length))
    model.add(LSTM(hp.Int('units', 64, 256, step=64)))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_tuned_model, objective='val_accuracy', max_epochs=20, directory='tuner_dir')
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val))